In [16]:
import pandas as pd
import numpy as np
import os
import re
import csv
pd.options.display.max_rows = 7000
pd.options.display.max_columns = 30

In [17]:
# read files, info needs to be matched for each
path_to_raw = '/Users/declanchin/Desktop/MEDSL/2018-precincts/precinct/MI/raw/'
all_files = [f for f in os.listdir(path_to_raw) if 'readme' not in f]
df_dic = {}
for file in all_files:
    if (file == '2020vote.txt') or (file == '2020name.txt') or (file == '2020offc.txt'): #read district as string
        df = pd.read_csv(path_to_raw+file, delimiter='\t', header=None, dtype={3:str})
        df = df[df.columns[:-1]]
        df = df.fillna('')
        df_dic[file] = df
    else:
        df = pd.read_csv(path_to_raw+file, delimiter='\t', header=None)
        df = df[df.columns[:-1]]
        df = df.fillna('')
        df_dic[file] = df

In [18]:
#assigning variables and column names (based on readme.txt)
votes=df_dic['2018vote.txt']
names=df_dic['2018name.txt']
counties=df_dic['county.txt']
offices=df_dic['2018offc.txt']
cities=df_dic['2018city.txt']
votes.columns = ['year','stage', 'office_code','district','special_code',
             'candidate','county_code', 'city_town_code','ward',
                 'precinct_number','precinct_label','votes']
offices.columns = ['year','stage', 'office_code','district','special_code',
                  'office_desc']
cities.columns = ['year','stage','county_code', 'city_town_code',
                 'city_town_desc']
cities = cities[['county_code', 'city_town_code','city_town_desc']]
counties.columns = ['county_code','county_name']
names.columns = ['year','stage', 'office_code','district',
                    'special_code','candidate_id','last','first','middle',
                   'party_detailed']


In [19]:
votes = votes.merge(offices, on = ['office_code','district','special_code','year','stage'], how='left')
# offices map (readme)
office_map = {0: 'POLL BOOK TOTALS (TOTAL VOTERS)',
            1: 'President of the United States',
           2: 'Governor',
           3: 'Secretary of State',
           4: 'Attorney General',
           5: 'United States Senator',
           6: 'U.S. Representative in Congress',
           7: 'State Senate',
           8: 'State Representative',
           9: 'Member of the State Board of Education',
          10: 'Member of the University of Michigan Board of Regents',
          11: 'Member of the Michigan State University Board of Trustees',
          12: 'Member of the Wayne State University Board of Governors',
          13: 'Justice of the Supreme Court',
        14: 'DISTRICT COURT OF APPEALS',
          90: 'Statewide Ballot Proposals'}
votes['office'] = votes['office_code'].replace(office_map)
votes['office'] = np.where(((votes['special_code']==1)|(votes['special_code']==2)|(votes['special_code']==3)),
                           votes['office_desc'],votes['office'])
#votes = votes.rename(columns = {'office_code':'office'})
# special map (readme)
special_map = {0:    'Regular Term',
          1:    'Non-Incumbent',
          2: 'Incumbent - Partial Term',
            3: 'Incumbent - Partial Term',
            4:  'Incumbent - Partial Term',
          5: 'Non-Incumbent - Partial Term',
            6: 'Non-Incumbent - Partial Term',
            7: 'Non-Incumbent - Partial Term',
          8:    'Partial Term',
          9: 'New Judgeship',
            10: 'New Judgeship'}
votes['special_map'] = votes['special_code'].replace(special_map)
# party map
party_map = dict(zip(names.candidate_id,names.party_detailed)) 
votes['party_detailed'] = votes['candidate'].replace(party_map).str.strip()
# candidate map
names['candidate'] = names['first'] + ' ' + names['middle'] + ' ' +names['last']
names['candidate'] = names['candidate'].str.replace('  ',' ', regex=True).str.strip().str.replace('.','',regex=True).str.replace(',','',regex=True)
candidate_map = dict(zip(names.candidate_id,names.candidate)) 
votes['candidate'] = votes['candidate'].replace(candidate_map)
# jurisdiction_name map
votes = votes.merge(cities, on=['county_code','city_town_code'], how='left')
#votes['jurisdiction_name'] = votes['jurisdiction_name'].replace(jurisdiction_name_map)

# county map
county_map = dict(zip(counties.county_code,counties.county_name)) 
votes['county_code'] = votes['county_code'].replace(county_map)
votes = votes.rename(columns = {'county_code':'county_name','city_town_desc':'jurisdiction_name'})

# precinct field
votes['precinct'] = np.where(votes['precinct_label']!='',
                            votes['precinct_number'].astype(str) + '-' + votes['precinct_label'],
                            votes['precinct_number'].astype(str))
votes['precinct'] = np.where(votes['ward']!=0,
                            votes['precinct'] + ', WARD ' + votes['ward'].astype(str),
                            votes['precinct'])
votes['precinct'] = votes['precinct'].str.strip()
# special field
votes['special'] = np.where(votes['special_map']=='Partial Term',
                           'TRUE', 'FALSE')

In [20]:
# new df with necessary columns to clean
df = votes[['precinct','office', 'year', 'stage','district','candidate','county_name',
           'jurisdiction_name','votes','party_detailed','special','special_code','office_code']]
df = df.applymap(lambda x:x.upper() if type(x) == str else x) 

In [21]:
# office fix
def fix_office(x):
    if x == 'POLL BOOK TOTALS (TOTAL VOTERS)': return 'BALLOTS CAST - TOTAL'
    if x == 'PRESIDENT OF THE UNITED STATES': return 'US PRESIDENT'
    if x == 'UNITED STATES SENATOR': return 'US SENATE'
    if x == 'U.S. REPRESENTATIVE IN CONGRESS': return 'US HOUSE'
    if x == 'STATE REPRESENTATIVE': return 'STATE HOUSE'
    else: return x
df['office']=df.office.apply(fix_office)

# district fix
df['district'] = df['district'].astype(str).str[:-2].str.zfill(3).replace('000','')
df['district'] = np.where(((df['district']=='')&(df['office']!='BALLOTS CAST - TOTAL')),
                         'STATEWIDE', df['district'])

def get_magnitude(x):
    if x == 'BALLOTS CAST - TOTAL': return 0
    if x in ['MEMBER OF THE STATE BOARD OF EDUCATION',
             'MEMBER OF THE UNIVERSITY OF MICHIGAN BOARD OF REGENTS',
             'MEMBER OF THE MICHIGAN STATE UNIVERSITY BOARD OF TRUSTEES',
             'MEMBER OF THE WAYNE STATE UNIVERSITY BOARD OF GOVERNORS']:
        return 2
    else: return 1
df['magnitude']=df.office.apply(get_magnitude)

def fix_party_detailed(x):
    if x == 'REP': return "REPUBLICAN"
    if x == 'DEM': return "DEMOCRAT"
    if x == 'LIB': return 'LIBERTARIAN'
    if x == 'GRN': return 'GREEN'
    if x == 'NLP': return 'NATURAL LAW PARTY'
    if x == 'UST': return 'US TAXPAYERS PARTY'
    if x == 'WCP': return 'WORKING CLASS PARTY'
    if x == 'NPA': return ''
    else: return x
df['party_detailed'] = df.party_detailed.apply(fix_party_detailed)
df['party_detailed'] = np.where((df['office']=='JUSTICE OF THE SUPREME COURT')|(df['office'].str.contains('STATE PROPOSAL|DISTRICT COURT')),
                               'NONPARTISAN',df['party_detailed'])

def get_party_simplified(x):
    if x in ['REPUBLICAN','DEMOCRAT','LIBERTARIAN','NONPARTISAN']: return x
    if x =='': return ''
    else: return 'OTHER'
df['party_simplified'] = df.party_detailed.apply(get_party_simplified)

# NPA is writein
df['writein'] = np.where((df['candidate']!='BALLOTS CAST - TOTAL')&(df['party_detailed']==''),
                        'TRUE','FALSE')

def get_dataverse(x):
    if x=='US PRESIDENT': return 'PRESIDENT'
    if x=='US HOUSE': return 'HOUSE'
    if x=='US SENATE': return 'SENATE'
    if x in ['STATE HOUSE','MEMBER OF THE STATE BOARD OF EDUCATION',
            'MEMBER OF THE UNIVERSITY OF MICHIGAN BOARD OF REGENTS',
             'MEMBER OF THE MICHIGAN STATE UNIVERSITY BOARD OF TRUSTEES',
            'MEMBER OF THE WAYNE STATE UNIVERSITY BOARD OF GOVERNORS',
            'JUSTICE OF THE SUPREME COURT', "STATE SENATE",
            'DISTRICT COURT OF APPEALS', "ATTORNEY GENERAL",
       'STATE PROPOSAL - 18-1: LEGISLATIVE INITIATIVE: COALITION TO REGULATE MARIJUANA LIKE ALCOHOL',
       'STATE PROPOSAL - 18-2: CONSTITUTIONAL AMENDMENT: VOTERS NOT POLITICIANS',
       'STATE PROPOSAL - 18-3: CONSTITUTIONAL AMENDMENT: PROMOTE THE VOTE',
            'GOVERNOR', 'SECRETARY OF STATE']: return 'STATE'
    if x == 'BALLOTS CAST - TOTAL': return ''
    else: return 'LOCAL'
df['dataverse'] = df.office.apply(get_dataverse)

def get_mode(x):
    if 'AVCB' in x: return 'ABSENTEE'
    else: return 'TOTAL'
df['mode'] = df.precinct.apply(get_mode)

In [22]:
# add year, stage, state, special, date, office
df['state'] = 'MICHIGAN'
df['date']= '2018-11-06'
df['readme_check'] = np.where(df['precinct']=='9999', 'TRUE','FALSE')
df['stage']='GEN'
#making counties with AVBM precincts non-absentee votes  "election day"
df['mode'] = np.where((df['county_name'].isin(['CASS', 'CLINTON', 'GLADWIN', 'LAPEER', 'ST. CLAIR', 'WAYNE']))&(df['mode']!='ABSENTEE'),
                     'ELECTION DAY',df['mode'])

# # drop writein candidates with 0 votes, (ACTUALLY DON't DUE TO DISCR. with official writein tallies)
# df = df[~((df['writein']=='TRUE')&(df['votes']==0))]

#add county fips
df['county_name'] = df['county_name'].replace('GD. TRAVERSE', 'GRAND TRAVERSE')
county_fips = pd.read_csv('/Users/declanchin/Desktop/MEDSL/2020-precincts/help-files/county-fips-codes.csv')
county_fips = county_fips[county_fips['state']=='Michigan'].drop(columns='state')
df=df.merge(county_fips, on='county_name', how='left')

In [23]:
# crosswalk to update jurisdiction fips file for mi
fips_file = pd.read_csv('/Users/declanchin/Desktop/MEDSL/2020-precincts/help-files/jurisdiction-fips-codes.csv', dtype={'jurisdiction_fips':str})
crosswalk = pd.read_excel('jurisdiction_crosswalk.xlsx')
crosswalk = crosswalk.rename(columns={'jurisdiction_name_file':'jurisdiction_name'})
fips_file['jurisdiction_fips'] = fips_file['jurisdiction_fips'].str.zfill(5)
fips_file['county_fips'] = fips_file['jurisdiction_fips'].apply(lambda fips: int(str(fips)[:5]))
fips_file = fips_file[fips_file['state']=='Michigan']
fips_file=fips_file.merge(crosswalk, on = ['state','county_fips','jurisdiction_name'], how='left')
fips_file['jurisdiction_name'] = np.where(fips_file['jurisdiction_name_raw'].notnull(), fips_file['jurisdiction_name_raw'], fips_file['jurisdiction_name'])
fips_file = fips_file.loc[:,['state','jurisdiction_name','jurisdiction_fips','county_fips']]
fips_file = fips_file.append([{'state':'Michigan','jurisdiction_name':'OVID CITY',
                             'jurisdiction_fips':2603761860,'county_fips':26037}, {'state':'Michigan','jurisdiction_name':'GARFIELD TOWNSHIP',
                             'jurisdiction_fips':2605531580,'county_fips':26055}],ignore_index=True)
fips_file['state'] = fips_file['state'].str.upper()
df = df.merge(fips_file, on=['state', 'county_fips', 'jurisdiction_name'], how="left")
df['jurisdiction_fips'] = df['jurisdiction_fips'].fillna(df['county_fips']).astype(int)
# state codes
state_codes = pd.read_csv('/Users/declanchin/Desktop/MEDSL/2020-precincts/help-files/merge_on_statecodes.csv')
state_codes = state_codes[state_codes['state']=='Michigan']
state_codes['state'] = state_codes['state'].str.upper()
df=df.merge(state_codes, on='state')

df = df.replace(np.nan, '')

### Calculating Discr. for README

In [24]:
official=pd.read_excel('2018GEN_MI_CENR_BY_COUNTY.xls')
official=official[['CountyName','StatusCode','CandidateID','CandidateVotes','OfficeCode(Text)','OfficeDescription']]
official.columns = ['county_name','special_code','candidate_id','votes_official','office_code','OfficeDescription']
official=official.dropna()
official['county_name'] = official['county_name'].replace('GD. TRAVERSE', "GRAND TRAVERSE")
official = official.applymap(lambda x:int(x) if type(x) == float else x) 

cand_ids=names.loc[:,['candidate','office_code','candidate_id','district','special_code']]
cand_ids['office']=cand_ids['office_code'].replace(office_map).str.upper()
cand_ids['office']= cand_ids.office.apply(fix_office)
cand_ids['district'] = cand_ids['district'].astype(str).str[:-2].str.zfill(3).replace('000','')
cand_ids['district'] = np.where(((cand_ids['district']=='')&(cand_ids['office']!='BALLOTS CAST - TOTAL')),
                         'STATEWIDE', cand_ids['district'])
cand_ids = cand_ids[['candidate','office','candidate_id','district','special_code','office_code']]
# merging on code info
our=df.loc[:,['candidate','office','county_name','votes','special_code','district']]
our['office'] = np.where(our['office'].str.contains('STATE PROPOSAL'), 'STATEWIDE BALLOT PROPOSALS',our['office'])
our=our.groupby(['office','county_name','candidate','district','special_code']).agg(sum).reset_index()
our['votes'] = our['votes'].fillna(0)
our = our.merge(cand_ids, on=['candidate','office','special_code','district'], how ='left').drop_duplicates()

# merging on code info
our_no_9999=df.loc[:,['candidate','office','county_name','votes','special_code','district','precinct']]
our_no_9999['office'] = np.where(our_no_9999['office'].str.contains('STATE PROPOSAL'), 'STATEWIDE BALLOT PROPOSALS',our_no_9999['office'])
our_no_9999 = our_no_9999[our_no_9999['precinct']!='9999']
our_no_9999=our_no_9999.groupby(['office','county_name','candidate','district','special_code']).agg(sum).reset_index().rename(columns = {'votes':'votes_no_9999'})
our_no_9999['votes_no_9999'] = our_no_9999['votes_no_9999'].fillna(0)
our_no_9999 = our_no_9999.merge(cand_ids, on=['candidate','office','special_code','district'], how ='left').drop_duplicates()

# no 9999
discr_no_9999=official.merge(our_no_9999, on  = ['county_name','special_code','candidate_id','office_code'], how = 'right')
discr_no_9999=discr_no_9999[discr_no_9999['office']!='BALLOTS CAST - TOTAL']
discr_no_9999['votes_official'] = discr_no_9999['votes_official'].astype(int)
discr_no_9999['discrepancy_no_9999 (official - ours)'] = discr_no_9999['votes_official'] - discr_no_9999['votes_no_9999']
# # with 9999
discr=official.merge(our, on  = ['county_name','special_code','candidate_id','office_code'], how = 'right')
discr=discr[discr['office']!='BALLOTS CAST - TOTAL']
discr['votes_official'] = discr['votes_official'].astype(int)
discr['discrepancy_w_9999 (official - ours)'] = discr['votes_official'] - discr['votes']

all_discrepancies=discr.merge(discr_no_9999,on = ['county_name', 'special_code', 'candidate_id', 'votes_official',
       'office_code', 'OfficeDescription', 'office', 'candidate', 'district'],
                              how = 'left')
all_discrepancies.to_csv('mi18-vote-discrepancies.csv',index=False)

In [25]:
# adding readme checks based on discrepancies documentation
df=df.merge(all_discrepancies.loc[:,['county_name', 'special_code', 'candidate','office_code','discrepancy_w_9999 (official - ours)','discrepancy_no_9999 (official - ours)']],
        on = ['county_name', 'special_code', 'candidate','office_code'],
        how='left')
df['discrepancy_no_9999 (official - ours)'] = df['discrepancy_no_9999 (official - ours)'].fillna(0).astype(int)
df['readme_check'] = np.where((df['discrepancy_no_9999 (official - ours)']!=0)&(df['office']!='DISTRICT COURT OF APPEALS'),
                               'TRUE',df['readme_check'])
df=df.loc[:,['precinct', 'office', 'year', 'stage', 'district', 'candidate',
       'county_name', 'jurisdiction_name', 'votes', 'party_detailed',
       'special', 'magnitude', 'writein',
       'party_simplified', 'dataverse', 'mode', 'state', 'date',
       'readme_check', 'county_fips', 'jurisdiction_fips', 'state_po',
       'state_fips', 'state_cen', 'state_ic']]

In [26]:
df.to_csv('2018-mi-precinct-general-updated.csv', index=False,quoting=csv.QUOTE_NONNUMERIC)